<a href="https://colab.research.google.com/github/dhruvchopra2003/Newspaper_ocr/blob/master/pytesseract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependancies

In [ ]:
from IPython import display

!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install tesseract-ocr-hin

display.clear_output()

!pip install roboflow
!pip install ultralytics==8.0.20
display.clear_output()

import ultralytics
ultralytics.checks()

from google.colab import drive
drive.mount('/content/gdrive')

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/107.7 GB disk)


Mounted at /content/gdrive


In [ ]:
import pytesseract as ps
import shutil
import os
import random
import cv2
from PIL import Image
import numpy as np
from PIL import Image, ImageDraw
from roboflow import Roboflow
from ultralytics import YOLO
from os import listdir

# Training the model

## Importing the data
Only for when we need to train a new model

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="qc1QSl8ZV1uoroE6bdlI")
project = rf.workspace("rscoe-m2bvv").project("newspaper-8hwch")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.134`


Extracting Dataset Version Zip to newspaper-1 in yolov8:: 100%|██████████| 295/295 [00:00<00:00, 1732.27it/s]


In [ ]:
! yolo task=segment mode=train model=yolov8s.pt data=/content/gdrive/MyDrive/Impact_OCR/data.yaml epochs=50 batch=8 optimizer='Adam' lr0=3E-3 project=/content/gdrive/MyDrive/Impact_OCR/training_results

## Testing model working


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/gdrive/MyDrive/Impact_OCR/training_results/train5/weights/best.pt")
results = model.predict(source = "/content/gdrive/MyDrive/Impact_OCR/datasets/valid/images", save_txt=True, save=True, save_crop=True, hide_labels=True, line_thickness=1)

In [ ]:
# Copying the predict folder into the drive, for further analysis
!cp -r /content/runs/detect/predict /content/gdrive/MyDrive/Impact_OCR/output/validation_results

# Image Preprocessing steps

In [ ]:
# Image Preprocessing Functions

def erode(image):
  # Creating a kernel
  kernel = np.ones((5,5), np.uint8)
  # Using openCV'2 erode
  image_erode = cv2.erode(image, kernel)

  # Saving and opening the file
  filename = 'image_eroded.jpg'
  cv2.imwrite(filename, image_erode)
  return filename

def dilate(image):
  # kernel3 = np.ones((3,7), np.uint8)
  kernel3 = np.ones((2,3), np.uint8)
  image_dilation = cv2.dilate(image, kernel3, iterations=1)
  filename = 'image_dilation.jpg'
  # Saving the image
  cv2.imwrite(filename, image_dilation)
  return filename

def log_transform(image):
  # Apply log transform.
  c = 255/(np.log(1 + np.max(image)))
  log_transformed = c * np.log(1 + image)
  # Specify the data type.
  log_transformed = np.array(log_transformed, dtype = np.uint8)
  filename = 'log_transformed.jpg'
  cv2.imwrite('log_transformed.jpg', log_transformed)

  return filename


# Retrieving from the model

In [ ]:
def get_ocr(model, path):
  model = YOLO(f"/content/gdrive/MyDrive/Impact_OCR/training_results/{model}/weights/best.pt")
  results = model.predict(source = path,
                          save_txt=True,
                          save=True,
                          save_crop=True,
                          hide_labels=True,
                          line_thickness=1)

In [ ]:
get_ocr("train3", "/content/test5.jpeg")

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs
To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
Results saved to runs/detect/predict3
1 label saved to runs/detect/predict3/labels


# Pipeline for the OCR

## OCR Pipeline

In [ ]:
# OCR function
def ocr_read(img_path):
  # info = ps.image_to_string(Image.open(img_path), lang='hin')
  info = ps.image_to_string(Image.open(img_path))
  print(info)

def get_boxes(img_path):
  json_save = model.predict(img_path, confidence=40, overlap=30).json()
  print(json_save)

  articles = json_save['predictions'][0]
  headlines = json_save['predictions'][1]

  model.predict(img_path, confidence=40, overlap=30).save("prediction.jpg")

  # model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json()

  return articles, headlines

def crop_img(img_path, result):
  x1 = result['x'] - result['width'] / 2
  x2 = result['x'] + result['width'] / 2
  y1 = result['y'] - result['height'] / 2
  y2 = result['y'] + result['height'] / 2

  image = Image.open(img_path)
  draw = ImageDraw.Draw(image)
  draw.rectangle([x1, y1, x2, y2], outline='red', width=2)
  image.save('image_with_bounding_box.jpg')

# Cropping the image
  cropped_region = image.crop((x1, y1, x2, y2))
  cropped_region.save('cropped_region.jpg')


def run_ocr(img_path):
  # Getting the bounding box coordinates from the model
  articles, headlines = get_boxes(img_path)

  article_img = crop_img(img_path, articles)
  article_text = ocr_read('cropped_region.jpg')

  head_img = crop_img(img_path, headlines)
  header_text = ocr_read('cropped_region.jpg')

  return header_text, article_text

# Testing

In [ ]:
# OCR function
def ocr_read(img_path):
  # info = ps.image_to_string(Image.open(img_path), lang='hin')
  info = ps.image_to_string(Image.open(img_path))
  print(info)

def get_headlines(img_path, i):
  path = f'/content/runs/detect/predict{i}/crops/headlines'


# Test pipeline 2

In [ ]:
# Headline sub-branch

def yolo_ocr(model, path):
  model = YOLO(f"/content/gdrive/MyDrive/Impact_OCR/training_results/{model}/weights/best.pt")
  results = model.predict(source = path,
                          save_txt=True,
                          save=True,
                          save_crop=True,
                          hide_labels=True,
                          line_thickness=1)
  # return results


def tesseract_read(img_path):
  # info = ps.image_to_string(Image.open(img_path), lang='hin')
  info = ps.image_to_string(Image.open(img_path))
  return info


def read_header(header_path):
  # header_without_extension = os.path.splitext(header_path)
  header = tesseract_read(header_path)

  header = header.replace('\n',' ')
  return header


def read_article(img_path):
  # without_extension = os.path.splitext(img_path)
  article = tesseract_read(img_path)
  article = article.replace("\n\n", "\n")
  article = article.replace("-\n", "")
  return article


def save_content(name, content, he_bo):
  """
    args:
      name: name of the file to be saved as
      content: Content to be put in the file
      he_bo: specify if the file is a header or body
    working:
      The content is saved in a folder with the name of the image as specified
      The folder contains two .txt files, -he of the header and -bo for the body
  """

  file_path = "/content/gdrive/MyDrive/Impact_OCR/output/test_images/" + name
  print("saving in ", file_path)
  try:
    os.mkdir(file_path)
  except:
    print("file found")

  file_name = f"{he_bo}.txt"
  fi = file_path + '/' + file_name

  with open(fi, "w") as file:
    file.write(content)
  print(f"content printed to {name}/{he_bo}.txt")


def get_name(path):

  image_name = os.path.basename(path)
  return image_name


def delete_runs_dir():
  path = "/content/runs"
  try:
    shutil.rmtree(path)
    # print("deleted")
  except:
    print("Error occured while deleting runs")


def execute():
  img_path = input("Enter path: ")
  model = "train3"

  image_name = get_name(img_path)

  yolo_ocr(model, img_path)

  # Extracting the headline image
  headline = read_header(f"/content/runs/detect/predict/crops/headlines/{image_name}")
  article_body = read_article(img_path)

  # Deleting the runs directory
  delete_runs_dir()

  save_content(image_name[:-3], headline, "headline")
  save_content(image_name[:-3], article_body, "article")

  print("executed perfectly!")

In [ ]:
execute()

Enter path: /content/122-20230822-2-Times City_3.jpg


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs
To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels


saving in  /content/gdrive/MyDrive/Impact_OCR/output/test_images/122-20230822-2-Times City_3.
content printed to 122-20230822-2-Times City_3./headline.txt
saving in  /content/gdrive/MyDrive/Impact_OCR/output/test_images/122-20230822-2-Times City_3.
file found
content printed to 122-20230822-2-Times City_3./article.txt
executed perfectly!


In [ ]:
name = "hello"
content = "Test 1234556"
save_content(name[:-2], content, "header")

saving in  /content/gdrive/MyDrive/Impact_OCR/output/test_images/hel
content printed to hel/header.txt


In [ ]:
import cv2
import matplotlib.pyplot as plt

# img_file = "/content/122-20230822-2-Times City_1.jpg"
img_file = "/content/122-20230822-2-Times City_3.jpg"
img = cv2.imread(img_file)

def display(im_path):
  dpi=80
  im_data = plt.imread(im_path)
  height, width, depth = im_data.shape

  figsize = width/float(dpi), height/float(dpi)

  fig = plt.figure(figsize=figsize)
  ax = fig.add_axes([0,0,1,1])

  ax.axis('off')
  ax.imshow(im_data, cmap='gray')

  plt.show()


In [ ]:
inverted_img = cv2.bitwise_not(img)
cv2.imwrite("temp/inverted.jpg", inverted_img)

True

In [ ]:
display("temp/inverted.jpg")

# Class OCRProcessor

In [ ]:
class OCRProcessor:
    def __init__(self, model_path):
        self.model = YOLO(model_path)

    def yolo_ocr(self, img_path):
        results = self.model.predict(
            source=img_path,
            save_txt=True,
            save=True,
            save_crop=True,
            hide_labels=True,
            # line_thickness=1
        )
        return results

    def tesseract_read(self, img_path):
        info = ps.image_to_string(Image.open(img_path))
        return info

class TextExtractor:
    @staticmethod
    def combine_sentences(sentence1, sentence2):
      words1 = sentence1.split()
      words2 = sentence2.split()

      # Find the common suffix length
      common_suffix_length = 0
      for i in range(1, min(len(words1), len(words2)) + 1):
          if words1[-i] == words2[-i]:
              common_suffix_length = i
          else:
              break

      # Combine sentences with newline if no common suffix
      if common_suffix_length == 0:
          combined_sentence = sentence1 + '\n' + sentence2
      else:
          combined_sentence = ' '.join(words1 + words2[:-common_suffix_length])

      return combined_sentence


    @staticmethod
    def read_header(header_path):
      folder_dir = header_path
      header = ""

      for images in os.listdir(folder_dir):
        if(images.endswith('.jpg')):
          temp_path = header_path + images
          temp = TextExtractor.tesseract_read(temp_path)
          temp = temp.replace('\n', " ")
          # header += temp
          header = TextExtractor.combine_sentences(header, temp)

        # header += '\n'
        # header = header.replace('\n', ' ')
      return header

    @staticmethod
    def read_article(img_path):
        article = TextExtractor.tesseract_read(img_path)
        article = article.replace('\n', " ")
        article = article.replace("\n\n", "\n")
        article = article.replace("-\n", "")
        return article

    @staticmethod
    def tesseract_read(img_path):
        info = ps.image_to_string(Image.open(img_path))
        return info

class FileHandler:
    @staticmethod
    def get_name(path):
        image_name = os.path.basename(path)
        return image_name

    @staticmethod
    def save_content(folder_path, file_name, content):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, "w") as file:
            file.write(content)
        print(f"Content saved to {file_path}")

    @staticmethod
    def create_directory(directory_path):
        try:
            os.mkdir(directory_path)
        except FileExistsError:
            print("Directory already exists")

    @staticmethod
    def delete_directory(directory_path):
        try:
            shutil.rmtree(directory_path)
        except Exception as e:
            print(f"Error occurred while deleting directory: {e}")

    @staticmethod
    def delete_runs_dir():
      path = "/content/runs"
      try:
        shutil.rmtree(path)
        print("deleted runs")
      except:
        print("Error occured while deleting runs")


def execute():
    img_path = input("Enter image path: ")
    model_path = "/content/gdrive/MyDrive/Impact_OCR/training_results/train5/weights/best.pt"

    image_name = FileHandler.get_name(img_path)

    ocr_processor = OCRProcessor(model_path)
    ocr_processor.yolo_ocr(img_path)

    headline = TextExtractor.read_header(f"/content/runs/detect/predict/crops/headlines/")
    # headline = TextExtractor.read_header(f"/content/runs/detect/predict/crops/headlines/{image_name}")
    article_body = TextExtractor.read_article(img_path)

    # FileHandler.delete_runs_dir()

    output_directory = f"/content/gdrive/MyDrive/Impact_OCR/output/test_images/{image_name[:-3]}"
    FileHandler.create_directory(output_directory)

    FileHandler.save_content(output_directory, "headline.txt", headline)
    FileHandler.save_content(output_directory, "article.txt", article_body)

    print("Execution completed.")

if __name__ == "__main__":
    execute()


Enter image path: 5.jpg


Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs
To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels


Directory already exists
Content saved to /content/gdrive/MyDrive/Impact_OCR/output/test_images/5./headline.txt
Content saved to /content/gdrive/MyDrive/Impact_OCR/output/test_images/5./article.txt
Execution completed.


In [ ]:
FileHandler.delete_runs_dir()

deleted runs
